In [20]:
from meteostream import GefsClient

In [21]:
client = GefsClient()

In [22]:
ds = client.get_gefs_data()

Downloaded gefs.wave.t12z.prob.global.0p25.f000.grib2 to tmp/grib/
Downloaded gefs.wave.t12z.prob.global.0p25.f006.grib2 to tmp/grib/
Downloaded gefs.wave.t12z.prob.global.0p25.f012.grib2 to tmp/grib/
Downloaded gefs.wave.t12z.prob.global.0p25.f018.grib2 to tmp/grib/
Downloaded gefs.wave.t12z.prob.global.0p25.f024.grib2 to tmp/grib/
Downloaded gefs.wave.t12z.prob.global.0p25.f030.grib2 to tmp/grib/
Downloaded gefs.wave.t12z.prob.global.0p25.f036.grib2 to tmp/grib/
Downloaded gefs.wave.t12z.prob.global.0p25.f042.grib2 to tmp/grib/
Downloaded gefs.wave.t12z.prob.global.0p25.f048.grib2 to tmp/grib/
Downloaded gefs.wave.t12z.prob.global.0p25.f054.grib2 to tmp/grib/
Downloaded gefs.wave.t12z.prob.global.0p25.f060.grib2 to tmp/grib/
Downloaded gefs.wave.t12z.prob.global.0p25.f066.grib2 to tmp/grib/
Downloaded gefs.wave.t12z.prob.global.0p25.f072.grib2 to tmp/grib/
Downloaded gefs.wave.t12z.prob.global.0p25.f078.grib2 to tmp/grib/
Downloaded gefs.wave.t12z.prob.global.0p25.f084.grib2 to tmp/g

NotADirectoryError: [Errno 20] Not a directory: 'tmp/grib/gefs.wave.t12z.prob.global.0p25.f240.grib2'